<a href="https://colab.research.google.com/github/Rexoes/-dev-3/blob/main/AUE_PSO_ANN_Hyperparameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import random

In [9]:
# 1. Veri Setini Oluştur ve Ön İşlemleri Yap
X, y = make_classification(n_samples=3000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# 2. Model Oluşturma Fonksiyonu (PSO'da optimize edilecek hiperparametreleri içerir)
def create_ann_model(num_layers, num_neurons, learning_rate, dropout_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# 3. PSO Parametreleri
num_particles = 20
num_iterations = 20
inertia_weight = 0.7
c1 = 1.5  # Kişisel en iyi katsayı
c2 = 1.5  # Global en iyi katsayı

# Hiperparametre sınırları (örn: [min, max])
hyperparam_bounds = {
    "num_layers": (1, 3),
    "num_neurons": (16, 128),
    "learning_rate": (1e-4, 1e-2),
    "dropout_rate": (0.1, 0.5)
}

# Rastgele hiperparametre seçme fonksiyonu
def random_hyperparameters():
    return {
        "num_layers": random.randint(hyperparam_bounds["num_layers"][0], hyperparam_bounds["num_layers"][1]),
        "num_neurons": random.randint(hyperparam_bounds["num_neurons"][0], hyperparam_bounds["num_neurons"][1]),
        "learning_rate": random.uniform(hyperparam_bounds["learning_rate"][0], hyperparam_bounds["learning_rate"][1]),
        "dropout_rate": random.uniform(hyperparam_bounds["dropout_rate"][0], hyperparam_bounds["dropout_rate"][1]),
    }

In [12]:
# Parçacık sınıfı, her bir parçacık bir hiperparametre seti olarak ele alınır
class Particle:
    def __init__(self):
        self.position = random_hyperparameters()
        self.velocity = {key: 0 for key in self.position}
        self.best_position = self.position.copy()
        self.best_score = float('inf')

    def update_velocity(self, global_best_position, w, c1, c2):
        for key in self.position:
            r1, r2 = random.random(), random.random()
            cognitive_velocity = c1 * r1 * (self.best_position[key] - self.position[key])
            social_velocity = c2 * r2 * (global_best_position[key] - self.position[key])
            self.velocity[key] = w * self.velocity[key] + cognitive_velocity + social_velocity

    def update_position(self):
        for key in self.position:
            if isinstance(self.position[key], int):
                self.position[key] = int(self.position[key] + self.velocity[key])
                # Sınırları kontrol et ve sınır dışı değerleri ayarla
                self.position[key] = max(hyperparam_bounds[key][0], min(hyperparam_bounds[key][1], self.position[key]))
            elif isinstance(self.position[key], float):
                self.position[key] += self.velocity[key]
                # Sınırları kontrol et ve sınır dışı değerleri ayarla
                self.position[key] = max(hyperparam_bounds[key][0], min(hyperparam_bounds[key][1], self.position[key]))

In [6]:
# 4. PSO Algoritmasını Çalıştır
particles = [Particle() for _ in range(num_particles)]
global_best_position = particles[0].position
global_best_score = float('inf')

for iteration in range(num_iterations):
    for particle in particles:
        # Modeli parçacığın hiperparametreleriyle oluştur
        model = create_ann_model(particle.position["num_layers"],
                                 particle.position["num_neurons"],
                                 particle.position["learning_rate"],
                                 particle.position["dropout_rate"])

        # Eğitimi hızlı gerçekleştirmek için 1 epoch kullanılabilir
        history = model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
        y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
        accuracy = accuracy_score(y_test, y_pred)
        score = 1 - accuracy  # Düşük skor daha iyi

        # Kişisel en iyi skorları güncelle
        if score < particle.best_score:
            particle.best_score = score
            particle.best_position = particle.position.copy()

        # Global en iyi skorları güncelle
        if score < global_best_score:
            global_best_score = score
            global_best_position = particle.position.copy()

    # Hız ve pozisyon güncelleme
    for particle in particles:
        particle.update_velocity(global_best_position, inertia_weight, c1, c2)
        particle.update_position()

    print(f"Iteration {iteration + 1}/{num_iterations}, Best Accuracy: {1 - global_best_score}")

10/10 [==============================] - 0s 1ms/step
Iteration 1/20, Best Accuracy: 0.8533333333333334
10/10 [==============================] - 0s 1ms/step
Iteration 2/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 3/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 4/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 5/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 2ms/step
Iteration 6/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 7/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 8/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 9/20, Best Accuracy: 0.8666666666666667
10/10 [==============================] - 0s 1ms/step
Iteration 10/20, Bes

In [8]:
# Adaptif c1 ve c2 başlangıç ve bitiş değerleri
c1_start, c1_end = 2.5, 0.5
c2_start, c2_end = 0.5, 2.5

# 4. PSO Algoritmasını Çalıştır
particles = [Particle() for _ in range(num_particles)]
global_best_position = particles[0].position
global_best_score = float('inf')

for iteration in range(num_iterations):
    # Adaptif c1 ve c2 değerleri
    c1 = c1_start - (iteration / num_iterations) * (c1_start - c1_end)
    c2 = c2_start + (iteration / num_iterations) * (c2_end - c2_start)

    for particle in particles:
        # Modeli parçacığın hiperparametreleriyle oluştur
        model = create_ann_model(particle.position["num_layers"],
                                 particle.position["num_neurons"],
                                 particle.position["learning_rate"],
                                 particle.position["dropout_rate"])

        # Eğitimi hızlı gerçekleştirmek için 1 epoch kullanılabilir
        history = model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
        y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
        accuracy = accuracy_score(y_test, y_pred)
        score = 1 - accuracy  # Düşük skor daha iyi

        # Kişisel en iyi skorları güncelle
        if score < particle.best_score:
            particle.best_score = score
            particle.best_position = particle.position.copy()

        # Global en iyi skorları güncelle
        if score < global_best_score:
            global_best_score = score
            global_best_position = particle.position.copy()

    # Hız ve pozisyon güncelleme
    for particle in particles:
        particle.update_velocity(global_best_position, inertia_weight, c1, c2)
        particle.update_position()

    print(f"Iteration {iteration + 1}/{num_iterations}, Best Accuracy: {1 - global_best_score}, c1: {c1}, c2: {c2}")

10/10 [==============================] - 0s 1ms/step
Iteration 1/20, Best Accuracy: 0.8666666666666667, c1: 2.5, c2: 0.5
10/10 [==============================] - 0s 1ms/step
Iteration 2/20, Best Accuracy: 0.8666666666666667, c1: 2.4, c2: 0.6
10/10 [==============================] - 0s 1ms/step
Iteration 3/20, Best Accuracy: 0.8666666666666667, c1: 2.3, c2: 0.7
10/10 [==============================] - 0s 1ms/step
Iteration 4/20, Best Accuracy: 0.8666666666666667, c1: 2.2, c2: 0.8
10/10 [==============================] - 0s 1ms/step
Iteration 5/20, Best Accuracy: 0.8666666666666667, c1: 2.1, c2: 0.9
10/10 [==============================] - 0s 1ms/step
Iteration 6/20, Best Accuracy: 0.8666666666666667, c1: 2.0, c2: 1.0
10/10 [==============================] - 0s 1ms/step
Iteration 7/20, Best Accuracy: 0.8666666666666667, c1: 1.9, c2: 1.1
10/10 [==============================] - 0s 1ms/step
Iteration 8/20, Best Accuracy: 0.8666666666666667, c1: 1.8, c2: 1.2
10/10 [=========================

KeyboardInterrupt: 

In [13]:
# Adaptif w, c1 ve c2 başlangıç ve bitiş değerleri
w_start, w_end = 0.9, 0.4
c1_start, c1_end = 2.5, 0.5
c2_start, c2_end = 0.5, 2.5

# 4. PSO Algoritmasını Çalıştır
particles = [Particle() for _ in range(num_particles)]
global_best_position = particles[0].position
global_best_score = float('inf')

for iteration in range(num_iterations):
    # Adaptif w, c1 ve c2 değerleri
    w = w_start - (iteration / num_iterations) * (w_start - w_end)
    c1 = c1_start - (iteration / num_iterations) * (c1_start - c1_end)
    c2 = c2_start + (iteration / num_iterations) * (c2_end - c2_start)

    for particle in particles:
        # Modeli parçacığın hiperparametreleriyle oluştur
        model = create_ann_model(particle.position["num_layers"],
                                 particle.position["num_neurons"],
                                 particle.position["learning_rate"],
                                 particle.position["dropout_rate"])

        # Eğitimi hızlı gerçekleştirmek için 1 epoch kullanılabilir
        history = model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
        y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
        accuracy = accuracy_score(y_test, y_pred)
        score = 1 - accuracy  # Düşük skor daha iyi

        # Kişisel en iyi skorları güncelle
        if score < particle.best_score:
            particle.best_score = score
            particle.best_position = particle.position.copy()

        # Global en iyi skorları güncelle
        if score < global_best_score:
            global_best_score = score
            global_best_position = particle.position.copy()

    # Hız ve pozisyon güncelleme
    for particle in particles:
        particle.update_velocity(global_best_position, w, c1, c2)
        particle.update_position()

    print(f"Iteration {iteration + 1}/{num_iterations}, Best Accuracy: {1 - global_best_score}, w: {w}, c1: {c1}, c2: {c2}")

29/29 [==============================] - 0s 1ms/step
Iteration 1/20, Best Accuracy: 0.9844444444444445, w: 0.9, c1: 2.5, c2: 0.5
29/29 [==============================] - 0s 1ms/step
Iteration 2/20, Best Accuracy: 0.9844444444444445, w: 0.875, c1: 2.4, c2: 0.6
29/29 [==============================] - 0s 1ms/step
Iteration 3/20, Best Accuracy: 0.9855555555555555, w: 0.85, c1: 2.3, c2: 0.7
29/29 [==============================] - 0s 1ms/step
Iteration 4/20, Best Accuracy: 0.9855555555555555, w: 0.8250000000000001, c1: 2.2, c2: 0.8
29/29 [==============================] - 0s 1ms/step
Iteration 5/20, Best Accuracy: 0.9855555555555555, w: 0.8, c1: 2.1, c2: 0.9
29/29 [==============================] - 0s 1ms/step
Iteration 6/20, Best Accuracy: 0.9855555555555555, w: 0.775, c1: 2.0, c2: 1.0
29/29 [==============================] - 0s 1ms/step
Iteration 7/20, Best Accuracy: 0.9855555555555555, w: 0.75, c1: 1.9, c2: 1.1
29/29 [==============================] - 0s 1ms/step
Iteration 8/20, Best Ac

In [14]:
# 5. En İyi Hiperparametre Seti ile Nihai Model
print("Optimal Hiperparametreler:", global_best_position)
best_model = create_ann_model(global_best_position["num_layers"],
                              global_best_position["num_neurons"],
                              global_best_position["learning_rate"],
                              global_best_position["dropout_rate"])
history = best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Optimal Hiperparametreler: {'num_layers': 2, 'num_neurons': 87, 'learning_rate': 0.0030795873224693095, 'dropout_rate': 0.354601565849784}
Epoch 1/50
66/66 [==============================] - 1s 5ms/step - loss: 0.2436 - accuracy: 0.9081 - val_loss: 0.0951 - val_accuracy: 0.9833
Epoch 2/50
66/66 [==============================] - 0s 3ms/step - loss: 0.1104 - accuracy: 0.9690 - val_loss: 0.0914 - val_accuracy: 0.9822
Epoch 3/50
66/66 [==============================] - 0s 3ms/step - loss: 0.1054 - accuracy: 0.9724 - val_loss: 0.0879 - val_accuracy: 0.9844
Epoch 4/50
66/66 [==============================] - 0s 3ms/step - loss: 0.1002 - accuracy: 0.9738 - val_loss: 0.0853 - val_accuracy: 0.9856
Epoch 5/50
66/66 [==============================] - 0s 3ms/step - loss: 0.0951 - accuracy: 0.9743 - val_loss: 0.0853 - val_accuracy: 0.9833
Epoch 6/50
66/66 [==============================] - 0s 3ms/step - loss: 0.0892 - accuracy: 0.9800 - val_loss: 0.0823 - val_accuracy: 0.9822
Epoch 7/50
66/66 [===